In [1]:
# 1. 기존 충돌 라이브러리 제거
!pip uninstall -y torch torchvision torchaudio transformers huggingface_hub

# 2. 호환되는 버전으로 재설치 (Colab A100 환경 기준)
# torch는 건드리지 않고(Colab 기본값 유지), transformers와 accelerator만 최신화합니다.
!pip install -q -U transformers accelerate bitsandbytes huggingface_hub

print("✅ 설치 완료! 이제 상단 메뉴 [런타임] > [세션 다시 시작]을 누르고 Step 2로 넘어가세요.")

Found existing installation: torch 2.9.0+cu126
Uninstalling torch-2.9.0+cu126:
  Successfully uninstalled torch-2.9.0+cu126
Found existing installation: torchvision 0.24.0+cu126
Uninstalling torchvision-0.24.0+cu126:
  Successfully uninstalled torchvision-0.24.0+cu126
Found existing installation: torchaudio 2.9.0+cu126
Uninstalling torchaudio-2.9.0+cu126:
  Successfully uninstalled torchaudio-2.9.0+cu126
Found existing installation: transformers 4.57.6
Uninstalling transformers-4.57.6:
  Successfully uninstalled transformers-4.57.6
Found existing installation: huggingface-hub 0.36.0
Uninstalling huggingface-hub-0.36.0:
  Successfully uninstalled huggingface-hub-0.36.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 141.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 915.7/915.7 MB 1.5 MB/s eta 0:00:00
   

In [3]:
import os
import re
import json
import torch
import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login

# =================================================================
# 1. 설정
# =================================================================
from google.colab import userdata
HF_TOKEN = userdata.get('HF_TOKEN')
CSV_PATH = "/content/drive/MyDrive/멋사 NLP 자연어처리/FinalProject/merged_lm_trend_20244_20253.csv"
SAVE_PATH = "/content/drive/MyDrive/멋사 NLP 자연어처리/FinalProject/망원_상권x매출json.json"


if HF_TOKEN == "hf_YourTokenHere":
    raise ValueError("❌ HF_TOKEN을 입력해주세요!")
else:
    login(token=HF_TOKEN)

# =================================================================
# 2. 분석 엔진
# =================================================================
class RobustMarketAnalyzer:
    def __init__(self, model_name="google/gemma-2-9b-it"):
        print(f"🚀 모델 로딩 중: {model_name} (A100 GPU)")
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, token=HF_TOKEN)
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="cuda:0",
            torch_dtype=torch.bfloat16,
            token=HF_TOKEN
        )
        self.device = "cuda:0"

    def _get_trend_str(self, group_df):
        trends = []
        for _, r in group_df.sort_values('기준_년분기_코드').iterrows():
            q = str(r['기준_년분기_코드'])
            store = int(r['store_sum'])
            # 매출 없으면 0원 대신 'N/A'로 표기해서 LLM이 헷갈리지 않게 함
            aov_str = f"{int(r['aov']):,}원" if pd.notna(r['aov']) else "정보없음"
            open_r = r['open_rate_mean']
            close_r = r['close_rate_mean']
            trends.append(f"[{q}] 점포{store}개(개업{open_r}%/폐업{close_r}%)|객단가:{aov_str}")
        return " -> ".join(trends)

    def _repair_json(self, text):
        text = re.sub(r'```json\s*', '', text)
        text = re.sub(r'```\s*', '', text)
        match = re.search(r'(\{[\s\S]*\})', text)
        if match: text = match.group(1)
        text = re.sub(r'\"\s*\"', '", "', text)
        text = re.sub(r'\}\s*\"', '}, "', text)
        if text.count('{') > text.count('}'):
            text += '}' * (text.count('{') - text.count('}'))
        return text

    def analyze_group(self, group_df):
        latest = group_df.sort_values('기준_년분기_코드').iloc[-1]
        trend_hist = self._get_trend_str(group_df)

        # -------------------------------------------------------
        # [핵심] 데이터 유무에 따른 동적 프롬프트 구성
        # -------------------------------------------------------
        has_sales_data = pd.notna(latest.get('aov'))

        if has_sales_data:
            sales_info = f"객단가 {int(latest['aov']):,}원 / 주말비중 {latest['주말금액의존도']:.1f}% / 시간대 {latest['시간대비중']}"
            cust_info = f"성별 {latest['성별비중']} / 연령 {latest['연령대비중']}"
            instruction_add = "모든 항목을 상세히 분석하세요."
        else:
            # 데이터가 비어있음을 명확히 알림
            sales_info = "[[데이터 없음]] (절대 추측 금지, '분석 불가'로 기재)"
            cust_info = "[[데이터 없음]] (절대 추측 금지, '분석 불가'로 기재)"
            instruction_add = "매출과 고객 데이터가 없습니다. 오직 '점포 수'와 '개폐업률'을 보고 시장의 경쟁 상황(Supply)만 진단하세요. 나머지는 '분석 불가'라고 적으세요."

        market_info = f"""
        총 점포 {int(latest['store_sum'])}개 (순증감 {int(latest['net'])})
        개업률 {latest['open_rate_mean']}% vs 폐업률 {latest['close_rate_mean']}%
        1년 추세: {trend_hist}
        """

        prompt = f"""
<start_of_turn>user
데이터를 분석하여 JSON 리포트를 작성하세요. 전략(Strategy)은 제외하세요.
{instruction_add}

[대상] {latest['행정동_코드_명']} {latest['상권_코드_명']} - {latest['서비스_업종_코드_명']}

[데이터]
1. 시장 활력(점포/개폐업): {market_info}
2. 수익성(매출): {sales_info}
3. 고객 특성: {cust_info}

[출력 양식]
{{
  "id": "{latest['상권_코드_명']}_{latest['서비스_업종_코드_명']}",
  "market_analysis": {{
    "phase": "성장/보합/과당경쟁/쇠퇴 중 택1 (점포수/개폐업률 기반 판단)",
    "trend_review": "시장 흐름 분석",
    "competition_intensity": "경쟁 강도"
  }},
  "revenue_analysis": {{
    "profit_structure": "수익 구조 (데이터 없으면 '분석 불가')",
    "peak_time_characteristics": "시간/요일 특성 (데이터 없으면 '분석 불가')"
  }},
  "customer_analysis": {{
    "primary_target": "핵심 타겟 (데이터 없으면 '분석 불가')",
    "demographic_insight": "성별/연령 해석 (데이터 없으면 '분석 불가')"
  }}
}}
<end_of_turn>
<start_of_turn>model
```json
"""
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.device)

        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=800,
                temperature=0.1, # 창의성 억제
                do_sample=True
            )

        result = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

        try:
            json_text = result.split("```json")[-1].split("```")[0]
            repaired_text = self._repair_json(json_text)

            try:
                data = json.loads(repaired_text)
            except:
                import ast
                data = ast.literal_eval(repaired_text)

            # -------------------------------------------------------
            # [후처리] 환각 강제 제거 (Python Logic)
            # -------------------------------------------------------
            # 데이터가 없는데 LLM이 실수로 채웠을 경우 강제로 덮어쓰기
            if not has_sales_data:
                data['revenue_analysis'] = "분석 불가 (데이터 부족)"
                data['customer_analysis'] = "분석 불가 (데이터 부족)"

            # 원본 데이터 삽입
            data['raw_data_context'] = {
                "trend_history": trend_hist,
                "metrics": {
                    "store_count": int(latest['store_sum']),
                    "net_change": int(latest['net']),
                    "close_rate": latest['close_rate_mean'],
                    "has_sales_data": bool(has_sales_data) # 데이터 유무 플래그
                }
            }
            return data
        except:
            return None

# =================================================================
# 3. 메인 실행
# =================================================================
def main():
    df = pd.read_csv(CSV_PATH)
    all_groups = df.groupby(['상권_코드_명', '서비스_업종_코드_명'])

    analyzer = RobustMarketAnalyzer()
    results = []

    print(f"\n🔥 전체 {len(all_groups)}개 상권 분석 시작 (결측치 방어 로직 적용)...\n")

    for name, group in tqdm(all_groups, desc="Analyzing"):
        try:
            res = analyzer.analyze_group(group)
            if res:
                results.append(res)
        except Exception as e:
            # print(f"⚠️ Error: {name} - {e}")
            continue

    with open(SAVE_PATH, "w", encoding="utf-8") as f:
        json.dump(results, f, indent=2, ensure_ascii=False)

    print(f"\n✅ 완료! {SAVE_PATH}")

if __name__ == "__main__":
    main()

🚀 모델 로딩 중: google/gemma-2-9b-it (A100 GPU)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]


🔥 전체 66개 상권 분석 시작 (결측치 방어 로직 적용)...



Analyzing: 100%|██████████| 66/66 [36:27<00:00, 33.14s/it]


✅ 완료! /content/drive/MyDrive/멋사 NLP 자연어처리/FinalProject/망원_상권x매출json.json


**Final 수정 하드코딩**

In [1]:
import json

# 1. 파일 로드
input_path = "/content/망원상권x매출json.json"  # 사용자가 업로드한 파일명
output_path = "/content/망원상권x매출market_analysis_final_corrected.json"

try:
    with open(input_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    print(f"📂 원본 데이터 로드 완료: {len(data)}개 항목")

    corrected_count = 0

    # 2. 하드코딩 수정 로직 적용
    for item in data:
        # [수정 1] ID 매칭 실패 건 (망원2동 마포구청역 7번 -> 마포구청역 7번)
        if "망원2동 마포구청역 7번" in item["id"]:
            old_id = item["id"]
            new_id = old_id.replace("망원2동 ", "") # 앞의 행정동 명칭 제거
            item["id"] = new_id
            print(f"  🔧 [ID 수정] {old_id} -> {new_id}")
            corrected_count += 1

        # [수정 2] 망리단길 패스트푸드점 (Net -1인데 성장기라고 한 것)
        if item["id"] == "망리단길_패스트푸드점":
            print(f"  🛡️ [환각 수정] {item['id']}: 성장 -> 쇠퇴기 강제 교정")
            item["market_analysis"]["phase"] = "과당경쟁/쇠퇴기 (Corrected)"
            item["market_analysis"]["trend_review"] += " (자동교정: 점포수 1개 감소 및 높은 폐업률(23%)로 인해 쇠퇴 징후가 뚜렷함)"
            corrected_count += 1

        # [수정 3] 망리단길 호프-간이주점 (Net -2인데 성장기라고 한 것)
        if item["id"] == "망리단길_호프-간이주점":
            print(f"  🛡️ [환각 수정] {item['id']}: 성장 -> 쇠퇴기 강제 교정")
            item["market_analysis"]["phase"] = "과당경쟁/쇠퇴기 (Corrected)"
            item["market_analysis"]["trend_review"] += " (자동교정: 점포수 2개 감소 및 폐업률 발생으로 시장 위축세임)"
            corrected_count += 1

    # 3. 결과 저장
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2, ensure_ascii=False)

    print(f"\n✅ 수정 완료! 총 {corrected_count}건의 오류를 바로잡았습니다.")
    print(f"💾 최종 파일 저장됨: {output_path}")

except Exception as e:
    print(f"❌ 오류 발생: {e}")

📂 원본 데이터 로드 완료: 66개 항목
  🔧 [ID 수정] 망원2동 마포구청역 7번_치킨전문점 -> 마포구청역 7번_치킨전문점
  🔧 [ID 수정] 망원2동 마포구청역 7번_커피-음료 -> 마포구청역 7번_커피-음료
  🛡️ [환각 수정] 망리단길_패스트푸드점: 성장 -> 쇠퇴기 강제 교정
  🛡️ [환각 수정] 망리단길_호프-간이주점: 성장 -> 쇠퇴기 강제 교정

✅ 수정 완료! 총 4건의 오류를 바로잡았습니다.
💾 최종 파일 저장됨: /content/망원상권x매출market_analysis_final_corrected.json
